In [1]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [2]:
pip install transformers[torch]

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
pip install accelerate

In [5]:
pip install huggingface_hub

## Steps

1. prepare dataset
2. load pretrained Tokenizer, call it with dataset -> encoding
3. build PyTorch Dataset with encodings
4. Load pretrained Model
5. a. Load Trainer and train it
   b. or use naive Pytorch training pipeline

## Pretrained model for patentability from Hugging face

In [6]:
model_name =  'distilbert-base-uncased' #'AI-Growth-Lab/PatentSBERTa'

## Import libraries and pretrained model

In [7]:
# Pretty print
from pprint import pprint
# Datasets load_dataset function
from datasets import load_dataset
# Transformers Autokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Standard PyTorch DataLoader
from torch.utils.data import DataLoader

from transformers import pipeline, Trainer, TrainingArguments

import numpy as np
import torch
import torch.nn.functional as F

from transformers import logging

logging.set_verbosity_warning()

from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Download Dataset (USPTO)

use the `load_dataset` function to load all the patent applications that were filed to the USPTO in January 2016. We specify the date ranges of the training and validation sets as January 1-21, 2016 and January 22-31, 2016, respectively.

In [8]:
dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather",
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-21',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-31',
)

print('Loading is done!')

The repository for HUPD/hupd contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HUPD/hupd.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Loading dataset with config: PatentsConfig(name='sample', version=0.0.0, data_dir='sample', data_files={'train': ['https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather']}, description='Patent data from January 2016, for debugging')


Using metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710


Reading metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710
Filtering train dataset by filing start date: 2016-01-01
Filtering train dataset by filing end date: 2016-01-21
Filtering val dataset by filing start date: 2016-01-22
Filtering val dataset by filing end date: 2016-01-31


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Loading is done!


In [9]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id'],
        num_rows: 16153
    })
    validation: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id'],
        num_rows: 9094
    })
})

In [10]:
train_dict = dataset_dict['train']
print(len(train_dict))
type(train_dict)

16153


datasets.arrow_dataset.Dataset

In [11]:
validation_dict = dataset_dict['validation']
print(len(validation_dict))

9094


In [12]:
train_dict[:1]

{'patent_number': ['13261748'],
 'decision': ['ACCEPTED'],
 'title': ['MINI-OPTICAL NETWORK TERMINAL (ONT)'],
 'abstract': ['The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.'],
 'claims': ['1. A compact optical network terminal, comprising: a first interface coupled to a communications network; a second interface coupled to a network client, wherein the second interface is a network connectivity dongle with an optical transceiver at one end; and a processor including a circuitry and a memory coupled

In [13]:
train_dict[0]['claims']

'1. A compact optical network terminal, comprising: a first interface coupled to a communications network; a second interface coupled to a network client, wherein the second interface is a network connectivity dongle with an optical transceiver at one end; and a processor including a circuitry and a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network thereby reducing the unnecessary splitting of equal upstream wavelengths to all the network clients in the network. 2. The optical network terminal of claim 1, wherein the first interface includes an optical module that receives optical signals via the optical fiber link and converts the optical signals to electrical signals. 3. The optical network terminal of claim 2, wherein the optical module is selectively configurable to support two or more of a broadband passive optical 

In [14]:
train_dict[0]['abstract']

'The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.'

In [15]:
# Print info about the sizes of the train and validation sets
print(f'Train dataset size: {dataset_dict["train"].shape}')
print(f'Validation dataset size: {dataset_dict["validation"].shape}')

Train dataset size: (16153, 14)
Validation dataset size: (9094, 14)


## Pre-Processing the data

the label-to-index mapping for the decision status field by assigning the decision status labels to the class indices.

Since we want a patentability score between 0 to 1, I will assign the labels to either 0 or 1.

Pending is 1 for this project.

In [16]:
# Label-to-index mapping for the decision status field
decision_to_str = {'REJECTED': 0, 'ACCEPTED': 1, 'PENDING': 1, 'CONT-REJECTED': 0, 'CONT-ACCEPTED': 1, 'CONT-PENDING': 1}

# Helper function
def map_decision_to_string(example):
    return {'decision': decision_to_str[example['decision']]}

re-label the decision status fields of the examples in the training and validation sets

In [17]:
# Re-labeling/mapping.
train_set = dataset_dict['train'].map(map_decision_to_string)
val_set = dataset_dict['validation'].map(map_decision_to_string)

Map:   0%|          | 0/16153 [00:00<?, ? examples/s]

Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

In [18]:
# testing
train_set[:1]

{'patent_number': ['13261748'],
 'decision': [1],
 'title': ['MINI-OPTICAL NETWORK TERMINAL (ONT)'],
 'abstract': ['The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.'],
 'claims': ['1. A compact optical network terminal, comprising: a first interface coupled to a communications network; a second interface coupled to a network client, wherein the second interface is a network connectivity dongle with an optical transceiver at one end; and a processor including a circuitry and a memory coupled to the f

the abstract section of the patent applications

In [19]:
# Focus on the abstract section and tokenize the text using the tokenizer.
_SECTION_ = 'abstract'

In [20]:
# Training set
train_set = train_set.map(
    lambda e: tokenizer((e[_SECTION_]), truncation=True, padding='max_length'),
    batched=True)

Map:   0%|          | 0/16153 [00:00<?, ? examples/s]

In [21]:
# Validation set
val_set = val_set.map(
    lambda e: tokenizer((e[_SECTION_]), truncation=True, padding='max_length'),
    batched=True)

Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

In [22]:
train_set[:1]

{'patent_number': ['13261748'],
 'decision': [1],
 'title': ['MINI-OPTICAL NETWORK TERMINAL (ONT)'],
 'abstract': ['The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.'],
 'claims': ['1. A compact optical network terminal, comprising: a first interface coupled to a communications network; a second interface coupled to a network client, wherein the second interface is a network connectivity dongle with an optical transceiver at one end; and a processor including a circuitry and a memory coupled to the f

the claims section of the patent applications

In [23]:
# Focus on the abstract section and tokenize the text using the tokenizer.
_SECTION1_ = 'claims'

In [24]:
# Training set
train_set = train_set.map(
    lambda e: tokenizer((e[_SECTION1_]), truncation=True, padding='max_length'),
    batched=True)

Map:   0%|          | 0/16153 [00:00<?, ? examples/s]

In [25]:
# Validation set
val_set = val_set.map(
    lambda e: tokenizer((e[_SECTION1_]), truncation=True, padding='max_length'),
    batched=True)

Map:   0%|          | 0/9094 [00:00<?, ? examples/s]

In [26]:
train_set[:1]

{'patent_number': ['13261748'],
 'decision': [1],
 'title': ['MINI-OPTICAL NETWORK TERMINAL (ONT)'],
 'abstract': ['The present invention relates to passive optical network (PON), and in particular, to an optical network terminal (ONT) in the PON system. In one embodiment, the optical network terminal includes a first interface coupled to a communications network, a second interface coupled to a network client and a processor including a memory coupled to the first interface and to the second interface, wherein the processor is capable of converting optical signals to electric signals, such that the network client can access the communications network.'],
 'claims': ['1. A compact optical network terminal, comprising: a first interface coupled to a communications network; a second interface coupled to a network client, wherein the second interface is a network connectivity dongle with an optical transceiver at one end; and a processor including a circuitry and a memory coupled to the f

In [27]:
# Set the format
train_set.set_format(type='torch',
    columns=['input_ids', 'attention_mask', 'decision'])

val_set.set_format(type='torch',
    columns=['input_ids', 'attention_mask', 'decision'])

## Dataloader to create the training set and validation set loaders

In [28]:
# train_dataloader and val_data_loader
train_dataloader = DataLoader(train_set, batch_size=16)
val_dataloader = DataLoader(val_set, batch_size=16)

In [29]:
# Get the next batch
batch = next(iter(train_dataloader))
# Print the ids
pprint(batch['input_ids'])
# Print the labels
pprint(batch['decision'])

tensor([[  101,  1015,  1012,  ...,     0,     0,     0],
        [  101,  1015,  1012,  ...,  1996,  3653,   102],
        [  101,  1015,  1012,  ..., 16726,  1996,   102],
        ...,
        [  101,  1015,  1012,  ...,  1012,  1996,   102],
        [  101,  1015,  1012,  ...,  2034, 28688,   102],
        [  101,  1015,  1012,  ...,  2000,  4366,   102]])
tensor([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1])


In [30]:
# Print the input and output shapes
input_shape = batch['input_ids'].shape
output_shape = batch['decision'].shape
print(f'Input shape: {input_shape}')
print(f'Output shape: {output_shape}')

Input shape: torch.Size([16, 512])
Output shape: torch.Size([16])


In [31]:
# A helper function that converts ids into tokens
def convert_ids_to_string(tokenizer, input):
    return ' '.join(tokenizer.convert_ids_to_tokens(input))

print an example in the batch

In [32]:
# Print the example
pprint(convert_ids_to_string(tokenizer,batch['input_ids'][1]))

('[CLS] 1 . a method comprising : using a first reader to take a first reading '
 'of an inherent disorder feature of a tag ; using at least a second reader to '
 'take at least a second reading of the inherent disorder feature of the tag ; '
 'matching the first reading with at least the second reading ; determining '
 'one or more acceptance criteria , wherein at least one of the acceptance '
 'criteria is based on whether the first reading and the second reading match '
 'within a pre ##de ##ter ##mined threshold ; accepting the tag if the '
 'acceptance criteria are met ; and recording a finger ##print for the tag if '
 'the tag was accepted . 2 . the method of claim 1 , wherein determining one '
 'or more acceptance criteria further comprises : determining an acceptance '
 'criterion based on an individual reading . 3 . the method of claim 2 , '
 'wherein determining an acceptance criterion based on an individual reading '
 'comprises determining an acceptance criterion based on a

## Tune the Model

In [33]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

optim = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):
    i =0
    for batch in train_dataloader:
        i+=1
        print(f'the batch size of 16: {i}')
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['decision'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

the batch size of 16: 1
the batch size of 16: 2
the batch size of 16: 3
the batch size of 16: 4
the batch size of 16: 5
the batch size of 16: 6
the batch size of 16: 7
the batch size of 16: 8
the batch size of 16: 9
the batch size of 16: 10
the batch size of 16: 11
the batch size of 16: 12
the batch size of 16: 13
the batch size of 16: 14
the batch size of 16: 15
the batch size of 16: 16
the batch size of 16: 17
the batch size of 16: 18
the batch size of 16: 19
the batch size of 16: 20
the batch size of 16: 21
the batch size of 16: 22
the batch size of 16: 23
the batch size of 16: 24
the batch size of 16: 25
the batch size of 16: 26
the batch size of 16: 27
the batch size of 16: 28
the batch size of 16: 29
the batch size of 16: 30
the batch size of 16: 31
the batch size of 16: 32
the batch size of 16: 33
the batch size of 16: 34
the batch size of 16: 35
the batch size of 16: 36
the batch size of 16: 37
the batch size of 16: 38
the batch size of 16: 39
the batch size of 16: 40
the batch

## Testing

In [36]:
# Get the next batch
batch = next(iter(val_dataloader))
# Print the ids
pprint(batch['input_ids'])
# Print the labels
pprint(batch['decision'])

tensor([[  101,  1015,  1012,  ..., 16503,  2063,   102],
        [  101,  1015,  1012,  ...,  3341,  2012,   102],
        [  101,  1015,  1011,  ...,  1012,  2861,   102],
        ...,
        [  101,  1015,  1011,  ...,  3012,  2978,   102],
        [  101,  1015,  1011,  ...,  2689,  5418,   102],
        [  101,  1015,  1012,  ..., 27983,  2638,   102]])
tensor([0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])


In [37]:
print(batch['input_ids'])

tensor([[  101,  1015,  1012,  ..., 16503,  2063,   102],
        [  101,  1015,  1012,  ...,  3341,  2012,   102],
        [  101,  1015,  1011,  ...,  1012,  2861,   102],
        ...,
        [  101,  1015,  1011,  ...,  3012,  2978,   102],
        [  101,  1015,  1011,  ...,  2689,  5418,   102],
        [  101,  1015,  1012,  ..., 27983,  2638,   102]])


In [38]:
batch_size = 16
model_cpu = model.cpu()
with torch.no_grad():
  outputs = model_cpu(batch['input_ids']).logits
  print(outputs)
  predictions = F.softmax(outputs, dim = 1)
  print(predictions)
  labels = torch.argmax(predictions, dim = 1)
  print(labels)
  print("--------")
  print(batch['decision'])
  print("--------")
  res = labels == batch['decision']
  print(res)
  print(res.sum() / batch_size)

tensor([[-1.4297,  1.2983],
        [-1.4307,  1.2990],
        [-1.4307,  1.2990],
        [-1.4307,  1.2990],
        [-1.4308,  1.2990],
        [-1.4307,  1.2990],
        [-1.4308,  1.2991],
        [-1.4304,  1.2988],
        [-1.4301,  1.2986],
        [-1.4307,  1.2989],
        [-1.4300,  1.2985],
        [-1.4305,  1.2989],
        [-1.4307,  1.2990],
        [-1.4307,  1.2989],
        [-1.4307,  1.2989],
        [-1.4307,  1.2990]])
tensor([[0.0613, 0.9387],
        [0.0612, 0.9388],
        [0.0612, 0.9388],
        [0.0612, 0.9388],
        [0.0612, 0.9388],
        [0.0612, 0.9388],
        [0.0612, 0.9388],
        [0.0613, 0.9387],
        [0.0613, 0.9387],
        [0.0613, 0.9387],
        [0.0613, 0.9387],
        [0.0613, 0.9387],
        [0.0612, 0.9388],
        [0.0613, 0.9387],
        [0.0612, 0.9388],
        [0.0612, 0.9388]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
--------
tensor([0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
--------
te

## Validation

In [39]:
accuracy = []
for batch in val_dataloader:
  with torch.no_grad():
    outputs = model_cpu(batch['input_ids']).logits
    predictions_batch = F.softmax(outputs, dim = 1)
    labels = torch.argmax(predictions_batch, dim = 1)
    acc = (labels == batch['decision']).numpy().sum() / batch_size
    print(f"batch_average_accuray: {acc}")
    accuracy.append(acc)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


batch_average_accuray: 0.75
batch_average_accuray: 0.75
batch_average_accuray: 1.0
batch_average_accuray: 0.75
batch_average_accuray: 0.9375
batch_average_accuray: 0.9375
batch_average_accuray: 1.0
batch_average_accuray: 0.9375
batch_average_accuray: 1.0
batch_average_accuray: 0.875
batch_average_accuray: 0.9375
batch_average_accuray: 0.75
batch_average_accuray: 0.9375
batch_average_accuray: 0.9375
batch_average_accuray: 0.9375
batch_average_accuray: 0.9375
batch_average_accuray: 1.0
batch_average_accuray: 0.9375
batch_average_accuray: 0.75
batch_average_accuray: 0.9375
batch_average_accuray: 0.8125
batch_average_accuray: 0.9375
batch_average_accuray: 0.875
batch_average_accuray: 0.9375
batch_average_accuray: 0.8125
batch_average_accuray: 0.875
batch_average_accuray: 0.875
batch_average_accuray: 0.75
batch_average_accuray: 0.875
batch_average_accuray: 0.6875
batch_average_accuray: 0.6875
batch_average_accuray: 0.9375
batch_average_accuray: 0.875
batch_average_accuray: 0.8125
batch_aver

In [40]:
print(f"average accuracy: {np.mean(accuracy)}")

average accuracy: 0.8896089630931459


## Save the tuned model in "saved" directory

In [41]:
save_directory = "saved"
tokenizer.save_pretrained(save_directory)
model_cpu.save_pretrained(save_directory)

In [42]:
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model_saved = AutoModelForSequenceClassification.from_pretrained(save_directory)

## Testing the saved model

In [43]:
with torch.no_grad():
  outputs = model_saved(batch['input_ids']).logits
  print(outputs)
  predictions = F.softmax(outputs, dim = 1)
  print(predictions)
  labels = torch.argmax(predictions, dim = 1)
  print(labels)
  print("--------")
  print(batch['decision'])
  print("--------")
  res = labels == batch['decision']
  print(res)
  print(res.sum() / batch_size)

tensor([[-1.4303,  1.2987],
        [-1.4306,  1.2989],
        [-1.4306,  1.2990],
        [-1.4307,  1.2990],
        [-1.4307,  1.2989],
        [-1.4307,  1.2990]])
tensor([[0.0613, 0.9387],
        [0.0613, 0.9387],
        [0.0613, 0.9387],
        [0.0612, 0.9388],
        [0.0612, 0.9388],
        [0.0612, 0.9388]])
tensor([1, 1, 1, 1, 1, 1])
--------
tensor([1, 1, 0, 1, 1, 1])
--------
tensor([ True,  True, False,  True,  True,  True])
tensor(0.3125)


## Share the tuned model to Hugging Face

In [47]:
model_cpu.push_to_hub("srikanth0008/tuned-for-patentability")


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/srikanth0008/tuned-for-patentability/commit/04329b3635d5b00bc94fe4bf81b258d285c40c6a', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='04329b3635d5b00bc94fe4bf81b258d285c40c6a', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
tokenizer.push_to_hub("srikanth0008/tuned-for-patentability")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/srikanth0008/tuned-for-patentability/commit/ab27c7a04cf7bcfea53aaab2ca63fec88410aeb5', commit_message='Upload tokenizer', commit_description='', oid='ab27c7a04cf7bcfea53aaab2ca63fec88410aeb5', pr_url=None, pr_revision=None, pr_num=None)